In [2]:
import os
#import ipdb

import pandas as pd
import matplotlib as mpl
from matplotlib import lines
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

EXP_NAME = "c10/10.138.120.66"
headers = ['time','start_transfer', 'latency','status','gateway']

client_data = pd.read_csv(EXP_NAME, sep=',')
client_data.columns= headers
client_data = client_data.set_index(pd.DatetimeIndex(client_data.time))
print(client_data)


                                                  time  start_transfer  \
time                                                                     
2019-01-21 14:36:20.858381  2019-01-21 14:36:20.858381        0.015758   
2019-01-21 14:37:21.380359  2019-01-21 14:37:21.380359        0.016210   
2019-01-21 14:38:22.622213  2019-01-21 14:38:22.622213        1.659397   
2019-01-21 14:39:21.116643  2019-01-21 14:39:21.116643        0.015520   
2019-01-21 14:40:21.115858  2019-01-21 14:40:21.115858        0.016735   
2019-01-21 14:41:20.888193  2019-01-21 14:41:20.888193        0.018435   
2019-01-21 14:42:21.095968  2019-01-21 14:42:21.095968        0.015225   
2019-01-21 14:43:20.984068  2019-01-21 14:43:20.984068        0.015734   
2019-01-21 14:44:20.778384  2019-01-21 14:44:20.778384        0.015857   
2019-01-21 14:45:21.599016  2019-01-21 14:45:21.599016        0.016815   
2019-01-21 14:46:21.125677  2019-01-21 14:46:21.125677        0.015878   
2019-01-21 14:47:20.931100  2019-01-21